# Sentinel-2 Sunglint Finder with Earth Engine Python API

This notebook shows how to find Sentinel-2 scenes over sunglint geometry using the Earth Engine Python API.  
Adjust the `bbox`, `start_date`, `end_date`, and other parameters as needed.


In [ ]:
# 1. Install and import Earth Engine Python API
# Uncomment the following line if running in Colab or a new environment:
# !pip install earthengine-api

import ee
import math

# Initialize the Earth Engine client. 
# If running for the first time, you may need to authenticate:
# ee.Authenticate()
ee.Initialize()
print('Earth Engine initialized.')


In [ ]:
# 2. Define parameters: bounding box, date range, sunglint threshold, and cloud filter

# -------------------------------------------------------------------
# Modify these variables as needed:
xmin, ymin, xmax, ymax = -80.0, 20.0, -75.0, 25.0  # Example bbox [lon_min, lat_min, lon_max, lat_max]
start_date = '2021-06-01'
end_date   = '2021-08-31'

sun_glint_threshold = 30  # Degrees; scenes with glint angle ≤ this will be selected.
max_cloud_cover = 20      # Maximum cloud percentage for initial filter (0-100)
# -------------------------------------------------------------------

# Create an Earth Engine geometry for the bounding box
bbox = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax])
aoi = ee.Geometry.Rectangle([-92.7, 27.2, -91.7, 28.2])  # offshore Louisiana



In [ ]:
# 3. Load Sentinel-2 L1C collection, filter by date, bounds, and cloud cover

s2_collection = (ee.ImageCollection('COPERNICUS/S2') \
                .filterDate('2023-01-01', '2023-01-31') \
                .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)))

print('Total S2 scenes in date/area filter:', s2_collection.size().getInfo())


s2_glint_scenes = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2023-01-01', '2023-01-31') \
    .filter(ee.Filter.gt('MEAN_SOLAR_ZENITH_ANGLE', 60)) \
    .filter(ee.Filter.lt('MEAN_INCIDENCE_ZENITH_ANGLE_B8', 10)) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover))

print('Total sunglint scenes in date/area filter:',s2_glint_scenes.size().getInfo())

In [ ]:
import geemap


In [ ]:
# 6. (Optional) Mask out non-water pixels using JRC Surface Water dataset
# Load JRC Global Surface Water Occurrence band (0-100%)
jrc = ee.Image('JRC/GSW1_2/GlobalSurfaceWater').select('occurrence')
water_mask = jrc.gt(50)  # Keep pixels that are water >50% of the year

def mask_non_water(img):
    return img.updateMask(water_mask)

s2_glint_water = s2_glint_scenes.map(mask_non_water)
print('Applied water mask: resulting scenes:', s2_glint_water.size().getInfo())


In [ ]:

# Visualize s2_glint_scenes on the interactive map using RGB bands
Map.centerObject(aoi, 8)
Map.addLayer(s2_glint_water.median(), vis_params, 'S2 Sunglint Scenes')
Map.addLayer(aoi, {}, 'AOI')
Map

In [ ]:
# 1. Get Sentinel-2 collection, filter cloud-free and time period
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2023-06-01', '2023-06-02') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .filter(ee.Filter.gt('MEAN_SOLAR_ZENITH_ANGLE', 60))  # sun low

# 2. Compute a sun glint index (green vs. SWIR) for each image
def add_glint_index(img):
    green = img.select('B3').divide(10000)  # convert SR to reflectance if needed
    swir = img.select('B11').divide(10000)
    sgi = green.subtract(swir).divide(green.add(swir)).rename('SGI')
    return img.addBands(sgi)
s2 = s2.map(add_glint_index)

# # 3. Estimate glint pixel fraction and filter
# def set_glint_frac(img):
#     mask = img.select('B11').gt(0.13)  # SWIR glint mask
#     stats = mask.reduceRegion(**{
#         'reducer': ee.Reducer.mean(),
#         'scale': 100,
#         'maxPixels':5e14
#     })
#     return img.set('glint_frac', stats.get('B11'))

# s2_stats = s2.map(set_glint_frac)
# high_glint = s2_stats.filter(ee.Filter.gt('glint_frac', 0.2))

# 4. Visualize the glint index of the top scene
# best = ee.Image(high_glint.first())
# Map.addLayer(best.select('SGI'), {'min':-1, 'max':1}, 'GlintIndex')
# Map.centerObject(aoi)


In [ ]:
print(s2.first().bandNames().getInfo())


In [ ]:
s2.size()

In [ ]:
Map = geemap.Map()
Map.addLayer(s2.median(), vis_params, 'Sentinel-2 RGB')
Map
